<a href="https://colab.research.google.com/github/adicu7/TelecomXChallenge/blob/main/ChallengeTelecom_Latam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests

url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

response = requests.get(url)
datajson = response.json()

df = pd.DataFrame(datajson)

df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


## **Explorar columnas, tipos de datos y significado**

In [13]:
# Dimensión del DataFrame
print("Dimensiones del DataFrame:", df.shape)

# Ver nombres de columnas y tipos de datos
print("Columnas y tipos de datos:")
print(df.dtypes)

# Valores nulos por columna
print("\nValores nulos:")
print(df.isnull().sum())

Dimensiones del DataFrame: (7267, 6)
Columnas y tipos de datos:
customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object

Valores nulos:
customerID    0
Churn         0
customer      0
phone         0
internet      0
account       0
dtype: int64


In [12]:
# Expandimos cada columna que contiene un diccionario
customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

df_flat = pd.concat([df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

df_flat.head(5)



,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


## Análisis de la estructura expandida
Estas son ahora columnas planas, lo que es ideal para análisis. Tienes 21 columnas, entre ellas:

### 🧍 Demográficas
gender: Género del cliente.

SeniorCitizen: Si es ciudadano mayor (probablemente 1 sí, 0 no).

Partner, Dependents: Si tiene pareja / dependientes.

tenure: Antigüedad como cliente (en meses, seguramente).

### 📞 Servicios contratados
PhoneService, MultipleLines: Teléfono básico y múltiples líneas.

InternetService: Tipo (DSL, Fiber optic, None).

OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies: Servicios digitales extra.

### 💰 Cuenta y facturación
Contract: Tipo de contrato (Month-to-month, One year, Two year).

PaperlessBilling: ¿Recibe factura digital?

PaymentMethod: Forma de pago.

Charges.Monthly: Cargos mensuales.

Charges.Total: Total pagado históricamente.

### 🎯 Target
Churn: Si el cliente se fue (Yes) o no (No).

## Comprobación de incoherencias en los datos: Limpieza y validación de datos

In [17]:
# Valores nulos por columna
df_flat.isnull().sum()


,0
customerID,0
Churn,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0


In [20]:
# Filas duplicadas

duplicados = df_flat.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")

Número de filas duplicadas: 0


In [21]:
df_flat.dtypes

,0
customerID,object
Churn,object
gender,object
SeniorCitizen,int64
Partner,object
Dependents,object
tenure,int64
PhoneService,object
MultipleLines,object
InternetService,object


In [22]:
# Revisión de valores categóricos
categoricas = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
               'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
               'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
               'PaperlessBilling', 'PaymentMethod', 'Churn']

for col in categoricas:
    print(f"\nValores únicos en {col}:")
    print(df_flat[col].unique())


Valores únicos en gender:
['Female' 'Male']

Valores únicos en Partner:
['Yes' 'No']

Valores únicos en Dependents:
['Yes' 'No']

Valores únicos en PhoneService:
['Yes' 'No']

Valores únicos en MultipleLines:
['No' 'Yes' 'No phone service']

Valores únicos en InternetService:
['DSL' 'Fiber optic' 'No']

Valores únicos en OnlineSecurity:
['No' 'Yes' 'No internet service']

Valores únicos en OnlineBackup:
['Yes' 'No' 'No internet service']

Valores únicos en DeviceProtection:
['No' 'Yes' 'No internet service']

Valores únicos en TechSupport:
['Yes' 'No' 'No internet service']

Valores únicos en StreamingTV:
['Yes' 'No' 'No internet service']

Valores únicos en StreamingMovies:
['No' 'Yes' 'No internet service']

Valores únicos en Contract:
['One year' 'Month-to-month' 'Two year']

Valores únicos en PaperlessBilling:
['Yes' 'No']

Valores únicos en PaymentMethod:
['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']

Valores únicos en Churn:
['No' 'Ye

## Analisis de problemas encontrados:
Charges.Total: Es tipo object en vez de ser tipo numérico.

Churn: tiene un valor vacio.

Además de eso no se encontraron valores nulos, ni filas duplicadas.

# Solución

In [23]:
# Verificar y eliminar filas con Churn vacío
print("Filas con Churn vacío:", df_flat[df_flat['Churn'] == ''].shape[0])

# Filtrar esas filas
df_flat = df_flat[df_flat['Churn'] != '']

Filas con Churn vacío: 224


In [24]:
# Convertimos la columna 'Charges.Total' a float64
df_flat['Charges.Total'] = pd.to_numeric(df_flat['Charges.Total'], errors='coerce')

# Verificamos el cambio
print(df_flat['Charges.Total'].dtype)

# Ver cuántos valores quedaron como NaN por errores de conversión
print("Valores nulos después de conversión:", df_flat['Charges.Total'].isnull().sum())

float64
Valores nulos después de conversión: 11
